<a href="https://colab.research.google.com/github/andreeas26/my-notes/blob/master/_notebooks/2021-03-02-basic-learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "A basic Learner"
- categories: [fastpages, jupyter, learner]


This is my first post on fastpages.

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 10.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 194kB 28.1MB/s 
     |████████████████████████████████| 1.2MB 16.2MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *
import pandas as pd

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [ ]:
labels_df = pd.read_csv(path/"labels.csv")
labels_df.head(3)

,name,label
0,train/3/7463.png,0
1,train/3/21102.png,0
2,train/3/31559.png,0


In [ ]:
path.ls()[0].ls()

(#2) [Path('/root/.fastai/data/mnist_sample/valid/7'),Path('/root/.fastai/data/mnist_sample/valid/3')]

In [ ]:
def load_img(img_path):
  img = Image.open(path/img_path)
  img = tensor(img).view(28*28).float()/255
  return img

train_df = labels_df.loc[labels_df['name'].str.contains('train'), :]
test_df = labels_df.loc[labels_df['name'].str.contains('valid'), :]

train_dst = [(load_img(row['name']), tensor(row['label'])) for _, row in train_df.iterrows()]
test_dst  = [(load_img(row['name']), tensor(row['label'])) for _, row in test_df.iterrows()]

In [ ]:
train_dst[0][0].shape, train_dst[0][1].shape, test_dst[0][0].shape, test_dst[0][1].shape

(torch.Size([784]), torch.Size([]), torch.Size([784]), torch.Size([]))

In [ ]:
class BasicLearner:
  __name__ = 'BasicLearner'
  __repr__ = basic_repr('dls,model,opt_func,loss_func,metrics')

  def __init__(self, dls, model, opt_func, loss_func, metrics):
    store_attr('dls,model,opt_func,loss_func,metrics')

  def fit(self, epochs=10, lr=1e-2):
    opt = self.opt_func(self.model.parameters(), lr=lr)
    for e in range(epochs):
      self.model.train()
      train_loss = []
      for x,y in self.dls.train:
        pred = self.model(x)
        # print(pred.shape, y.shape)
        loss = self.loss_func(pred, y)
        train_loss.append(loss)
        opt.zero_grad()
        loss.backward()
        opt.step()
      train_loss = tensor(train_loss)

      self.model.eval()
      val_loss = []
      val_metrics = []
      for x,y in self.dls.valid:
        pred = self.model(x)
        loss = self.loss_func(pred, y)
        val_loss.append(loss)
        m = self.metrics(pred, y)
        val_metrics.append(m)

      val_loss = tensor(val_loss)
      val_metrics = tensor(val_metrics)

      msg = f"Epoch {e}/{epochs}: " \
        f"Train loss {train_loss.mean():.4f} " \
        f"Valid loss {val_loss.mean():.4f} " \
        f"{self.metrics.__name__} {val_metrics.mean():.2f}"
      print(msg)

In [ ]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

def batch_accuracy(predictions, targets):
    predictions = predictions.sigmoid()
    correct = (predictions > 0.5) == targets
    return correct.float().mean()

In [ ]:
class BasicOptim:
  __name__ = "BasicOptim"
  __repr__ = basic_repr('parameters,lr')
  def __init__(self, parameters, lr): self.parameters, self.lr = list(parameters), lr
  
  def step(self, *args, **kwargs):
    for p in self.parameters: p.data -= p.grad * self.lr
  
  def zero_grad(self, *args, **kwargs):
    for p in self.parameters: p.grad = None

In [ ]:
bs = 128
train_dl = DataLoader(train_dst, batch_size=bs)
test_dl = DataLoader(test_dst, batch_size=bs, shuffle=False)
dls = DataLoaders(train_dl, test_dl)

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
)

In [ ]:
learn = BasicLearner(dls, simple_net, BasicOptim, mnist_loss, batch_accuracy)
print(learn)

BasicLearner(dls=<fastai.data.core.DataLoaders object at 0x7fb68a127c50>, model=Sequential(
  (0): Linear(in_features=784, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=1, bias=True)
), opt_func=<class '__main__.BasicOptim'>, loss_func=<function mnist_loss at 0x7fb68a07a5f0>, metrics=<function batch_accuracy at 0x7fb68a07a170>)


In [ ]:
# help(torch.unsqueeze)

In [ ]:
learn.fit(10)

Epoch 0/10: Train loss 0.0530 Valid loss 0.0643 batch_accuracy 0.95
Epoch 1/10: Train loss 0.0503 Valid loss 0.0617 batch_accuracy 0.96
Epoch 2/10: Train loss 0.0480 Valid loss 0.0596 batch_accuracy 0.96
Epoch 3/10: Train loss 0.0460 Valid loss 0.0578 batch_accuracy 0.96
Epoch 4/10: Train loss 0.0444 Valid loss 0.0563 batch_accuracy 0.96
Epoch 5/10: Train loss 0.0429 Valid loss 0.0550 batch_accuracy 0.96
Epoch 6/10: Train loss 0.0416 Valid loss 0.0538 batch_accuracy 0.96
Epoch 7/10: Train loss 0.0404 Valid loss 0.0528 batch_accuracy 0.96
Epoch 8/10: Train loss 0.0394 Valid loss 0.0519 batch_accuracy 0.96
Epoch 9/10: Train loss 0.0384 Valid loss 0.0510 batch_accuracy 0.96
